In [3]:
import pandas as pd
import numpy as np

In [123]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [101]:
df_train = pd.read_csv("C:\\git\\dados\\nlp\\kaggle-sentiment\\train.csv", encoding="latin1")
df_train["text"] = df_train["text"].replace({np.nan: ""})
df_train.shape

(27481, 10)

In [103]:
df_test = pd.read_csv("C:\\git\\dados\\nlp\\kaggle-sentiment\\test.csv", encoding="latin1")
df_test["text"] = df_test["text"].replace({np.nan: ""})
df_test.shape

(4815, 9)

In [105]:
df = pd.concat( [df_train, df_test] )

In [107]:
df["output"] = df["sentiment"].replace({"neutral": 0, "negative": 0, "positive": 1})

C:\Users\anton\AppData\Local\Temp\ipykernel_26144\1959351941.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["output"] = df["sentiment"].replace({"neutral": 0, "negative": 0, "positive": 1})


In [109]:
vetorizador = CountVectorizer(max_features=1000, binary=True, lowercase=True)
# deixar 5 
# raiva 1
# [0, 0, 0, 0, 0, 0, 1]      raiva  - 1
# [0, 0, 1, 0, 0, 0, 0]      deixar - 5 

In [111]:
bow = vetorizador.fit_transform( df["text"] )


In [112]:
vocabulario = vetorizador.get_feature_names_out()

In [125]:
features = len(vocabulario)
features

1000

In [115]:
all_X = pd.DataFrame.sparse.from_spmatrix( bow, columns=vocabulario ).astype(np.float32)

In [117]:
train_X = all_X[:df_train.shape[0]]
test_X = all_X[df_train.shape[0]:]

In [119]:
train_Y = df["output"][:df_train.shape[0]]
test_Y = df["output"][df_train.shape[0]:]

In [127]:
modelo = Sequential()
modelo.add( Input( (features, ) ) )
modelo.add( Dense( 128, activation="relu" ) )
modelo.add( Dense( 16, activation="relu" ) )
modelo.add( Dense( 1, activation="sigmoid" ) )
modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         128,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │           2,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 130,209 (508.63 KB)

 Trainable params: 130,209 (508.63 KB)

 Non-trainable params: 0 (0.00 B)

In [129]:
modelo.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

In [131]:
historico = modelo.fit(train_X, train_Y, epochs=10, batch_size=32)

Epoch 1/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7755 - loss: 0.4963
Epoch 2/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8554 - loss: 0.3638
Epoch 3/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8710 - loss: 0.3352
Epoch 4/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8772 - loss: 0.3200
Epoch 5/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8847 - loss: 0.3036
Epoch 6/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8925 - loss: 0.2894
Epoch 7/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8981 - loss: 0.2769
Epoch 8/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9020 - loss: 0.2670
Epoch 9/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9097 - loss: 0.2560
Epoch 10/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9135 - loss: 0.2427


In [133]:
modelo.evaluate( test_X, test_Y )

151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8103 - loss: nan


[nan, 0.6191069483757019]

In [139]:
predict_vectorizer = CountVectorizer(max_features=features, binary=True, 
                                     lowercase = True, vocabulary=vocabulario)

In [155]:
texto = "I love traffic"
bow_predict = predict_vectorizer.fit_transform([texto])
predict_X = pd.DataFrame.sparse.from_spmatrix( bow_predict, columns=vocabulario ).astype(np.float32)

In [157]:
"negativo" if modelo.predict( predict_X )[0][0] < 0.5 else "positivo"

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


'positivo'